Before starting we need to import necessary libraries

In [1]:
from __future__ import print_function
import os
import argparse
import sys
sys.argv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torchvision import datasets, transforms
import random
import numpy as np

To ensure we get reproducible results we set the random seed for Python, Numpy and PyTorch.

In [2]:
SEED=1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark = False

In [3]:
parser = argparse.ArgumentParser(description='PyTorch CIFAR 10 Training')
parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                    help='input batch size for training (default: 128)')
parser.add_argument('--test-batch-size', type=int, default=128, metavar='N',
                    help='input batch size for testing (default: 128)')
parser.add_argument('--epochs', type=int, default=76, metavar='N',
                    help='number of epochs to train')
parser.add_argument('--weight-decay', '--wd', default=2e-4,
                    type=float, metavar='W')
parser.add_argument('--lr', type=float, default=0.1, metavar='LR',
                    help='learning rate')
parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                    help='SGD momentum')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--epsilon', default=0.031,
                    help='perturbation')
parser.add_argument('--num-steps', default=10,
                    help='perturb number of steps')
parser.add_argument('--step-size', default=0.007,
                    help='perturb step size')
parser.add_argument('--beta', default=6.0,
                    help='regularization, i.e., 1/lambda in TRADES')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=100, metavar='N',
                    help='how many batches to wait before logging training status')
parser.add_argument('--model-dir', default='./model-cifar-wideResNet',
                    help='directory of model for saving checkpoint')
parser.add_argument('--save-freq', '-s', default=1, type=int, metavar='N',
                    help='save frequency')

args, unknown = parser.parse_known_args()

In [4]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion * planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion * planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=100):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])
def test():
    net = ResNet18()
    y = net(torch.randn(1, 3, 32, 32))
    print(y.size())

In [5]:
use_cuda = not args.no_cuda and torch.cuda.is_available()
torch.manual_seed(args.seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

1.  first we need to load the last checkpoint:

In [6]:
net = ResNet18()
net.load_state_dict(torch.load('model-resnet-epoch76.pt'))
net.to(device) 

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

2.  Applying Auto attack to dataset and evaluate our trained model:




*  in order to apply auto attack, we need to get all of required packages from Auto Attack github

In [7]:
!pip install git+https://github.com/fra31/auto-attack

  Cloning https://github.com/fra31/auto-attack to c:\users\zahra\appdata\local\temp\pip-req-build-th880dno
  Resolved https://github.com/fra31/auto-attack to commit a39220048b3c9f2cca9a4d3a54604793c68eca7e
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/fra31/auto-attack 'C:\Users\zahra\AppData\Local\Temp\pip-req-build-th880dno'


*  Applying AA attack for Linf setting and calculating Adversarial attack rate

In [22]:
from autoattack import AutoAttack
adversary = AutoAttack(net, norm='Linf', eps=0.031, version='standard')
transform_test = transforms.Compose([
    transforms.ToTensor(),
])
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)
l = [x for (x, y) in test_loader]
x_test = torch.cat(l, 0)
l = [y for (x, y) in test_loader]
y_test = torch.cat(l, 0)
adv_complete = adversary.run_standard_evaluation(x_test, y_test,bs=500)

setting parameters for standard version
Files already downloaded and verified
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 92.76%
apgd-ce - 1/19 - 500 out of 500 successfully perturbed
apgd-ce - 2/19 - 498 out of 500 successfully perturbed
apgd-ce - 3/19 - 500 out of 500 successfully perturbed
apgd-ce - 4/19 - 500 out of 500 successfully perturbed
apgd-ce - 5/19 - 499 out of 500 successfully perturbed
apgd-ce - 6/19 - 500 out of 500 successfully perturbed
apgd-ce - 7/19 - 499 out of 500 successfully perturbed
apgd-ce - 8/19 - 500 out of 500 successfully perturbed
apgd-ce - 9/19 - 500 out of 500 successfully perturbed
apgd-ce - 10/19 - 498 out of 500 successfully perturbed
apgd-ce - 11/19 - 499 out of 500 successfully perturbed
apgd-ce - 12/19 - 500 out of 500 successfully perturbed
apgd-ce - 13/19 - 500 out of 500 successfully perturbed
apgd-ce - 14/19 - 500 out of 500 successfully perturbed
apgd-ce - 15/19 - 496 out of 500 successfully perturbed
a

In [23]:
adv_loader_Linf = torch.utils.data.DataLoader(adv_complete, batch_size=100, shuffle=False, num_workers=2)
num_total_images = 0
num_successful_attacks = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
    for adv_images in adv_loader_Linf:
        adv_images = adv_images.to(device)
        outputs_adv = net(adv_images)
        _, predicted_adv = torch.max(outputs_adv.data, 1)

        # Count the number of adversarial examples that were successfully attacked
        for i in range(len(images)):
            if predicted[i] != predicted_adv[i]:
                num_successful_attacks += 1
        
        # Increment the total number of images
        num_total_images += len(images)

# Calculate the adversarial attack rate
attack_rate = (num_successful_attacks / num_total_images) * 100
print(f"Adversarial attack rate for Linf norm: {attack_rate:.2f}%")


Adversarial attack rate for Linf norm: 90.02%


*  Applying AA attack for L2 setting and calculating Adversarial attack rate

In [9]:
from autoattack import AutoAttack
adversary = AutoAttack(net, norm='L2', eps=0.031, version='standard')
transform_test = transforms.Compose([
    transforms.ToTensor(),
])
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)
l = [x for (x, y) in test_loader]
x_test = torch.cat(l, 0)
l = [y for (x, y) in test_loader]
y_test = torch.cat(l, 0)
adv_complete_L2 = adversary.run_standard_evaluation(x_test, y_test,bs=500)

setting parameters for standard version


100%|██████████████████████████████████████████████████████████████| 170498071/170498071 [00:09<00:00, 17305109.94it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 92.76%
apgd-ce - 1/19 - 52 out of 500 successfully perturbed
apgd-ce - 2/19 - 40 out of 500 successfully perturbed
apgd-ce - 3/19 - 42 out of 500 successfully perturbed
apgd-ce - 4/19 - 50 out of 500 successfully perturbed
apgd-ce - 5/19 - 63 out of 500 successfully perturbed
apgd-ce - 6/19 - 43 out of 500 successfully perturbed
apgd-ce - 7/19 - 49 out of 500 successfully perturbed
apgd-ce - 8/19 - 41 out of 500 successfully perturbed
apgd-ce - 9/19 - 45 out of 500 successfully perturbed
apgd-ce - 10/19 - 53 out of 500 successfully perturbed
apgd-ce - 11/19 - 59 out of 500 successfully perturbed
apgd-ce - 12/19 - 55 out of 500 successfully perturbed
apgd-ce - 13/19 - 48 out of 500 successfully perturbed
apgd-ce - 14/19 - 40 out of 500 successfully perturbed
apgd-ce - 15/19 - 45 out of 500 successfully perturbed
apgd-ce - 16/19 - 54 out of 500 successfull

KeyboardInterrupt: 

In [ ]:
adv_loader_L2 = torch.utils.data.DataLoader(adv_complete_L2, batch_size=100, shuffle=False, num_workers=2)
num_total_images = 0
num_successful_attacks = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
    for adv_images in adv_loader_L2:
        adv_images = adv_images.to(device)
        outputs_adv = net(adv_images)
        _, predicted_adv = torch.max(outputs_adv.data, 1)

        # Count the number of adversarial examples that were successfully attacked
        for i in range(len(images)):
            if predicted[i] != predicted_adv[i]:
                num_successful_attacks += 1
        
        # Increment the total number of images
        num_total_images += len(images)

# Calculate the adversarial attack rate
attack_rate = (num_successful_attacks / num_total_images) * 100
print(f"Adversarial attack rate for L2 norm: {attack_rate:.2f}%")


3.  Anomaly detection:

*  to apply our anomally detection algorithm we download the packages from github

In [1]:
!git clone https://github.com/talreiss/Mean-Shifted-Anomaly-Detection.git
%cd Mean-Shifted-Anomaly-Detection

Cloning into 'Mean-Shifted-Anomaly-Detection'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 55 (delta 4), reused 4 (delta 3), pack-reused 46
Unpacking objects: 100% (55/55), 17.41 KiB | 1.34 MiB/s, done.
/content/Mean-Shifted-Anomaly-Detection


*  we download the required packages and install necessary libraries

In [2]:
!virtualenv venv --python python3
!source venv/bin/activate
!pip install -r requirements.txt --find-links https://download.pytorch.org/whl/torch_stable.html
!pip install faiss_gpu
import os

os.mkdir('data')

/bin/bash: virtualenv: command not found
/bin/bash: venv/bin/activate: No such file or directory
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 75.0 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement pkg-resources==0.0.0 (from versions: none)
ERROR: No matching distribution found for pkg-resources==0.0.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 12.1 MB/s eta 0:00:00


*  inorder to train a anomaly detection model for label 0 we can run following code

In [3]:
!python main.py --dataset=cifar10 --label=0 --backbone=18

Dataset: cifar10, Normal Label: 0, LR: 1e-05
cuda:0
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100% 44.7M/44.7M [00:00<00:00, 334MB/s]
100% 170498071/170498071 [00:13<00:00, 12727458.52it/s]
Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified
Files already

*  we need to seperate the dataset for different labels

In [7]:
import torchvision
import torchvision.transforms as transforms
import torch
# Define a transform to normalize the data
transform_test = transforms.Compose([
    transforms.ToTensor(),
])

# Download and load the CIFAR-10 dataset
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=1)

# Create empty lists to hold datasets for each class
airplane_dataset = []
automobile_dataset = []
bird_dataset = []
cat_dataset = []
deer_dataset = []
dog_dataset = []
frog_dataset = []
horse_dataset = []
ship_dataset = []
truck_dataset = []

# Loop over the entire training set and add each image to its respective class dataset
for batch_idx, (data, target) in enumerate(test_loader):
    for i in range(len(data)):
        image, label = data[i], target[i]
        if label == 0:
            airplane_dataset.append((image, label))
        elif label == 1:
            automobile_dataset.append((image, label))
        elif label == 2:
            bird_dataset.append((image, label))
        elif label == 3:
            cat_dataset.append((image, label))
        elif label == 4:
            deer_dataset.append((image, label))
        elif label == 5:
            dog_dataset.append((image, label))
        elif label == 6:
            frog_dataset.append((image, label))
        elif label == 7:
            horse_dataset.append((image, label))
        elif label == 8:
            ship_dataset.append((image, label))
        elif label == 9:
            truck_dataset.append((image, label))

# Create separate data loaders for each class dataset
airplane_loader = torch.utils.data.DataLoader(airplane_dataset, batch_size=64, shuffle=True)
automobile_loader = torch.utils.data.DataLoader(automobile_dataset, batch_size=64, shuffle=True)
bird_loader = torch.utils.data.DataLoader(bird_dataset, batch_size=64, shuffle=True)
cat_loader = torch.utils.data.DataLoader(cat_dataset, batch_size=64, shuffle=True)
deer_loader = torch.utils.data.DataLoader(deer_dataset, batch_size=64, shuffle=True)
dog_loader = torch.utils.data.DataLoader(dog_dataset, batch_size=64, shuffle=True)
frog_loader = torch.utils.data.DataLoader(frog_dataset, batch_size=64, shuffle=True)
horse_loader = torch.utils.data.DataLoader(horse_dataset, batch_size=64, shuffle=True)
ship_loader = torch.utils.data.DataLoader(ship_dataset, batch_size=64, shuffle=True)
truck_loader = torch.utils.data.DataLoader(truck_dataset, batch_size=64, shuffle=True)


100%|██████████| 169001437/169001437 [00:13<00:00, 12675639.59it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data


*  we apply auto attack to dataset with our desired label

In [17]:
class AdversarialDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        return sample, label

l = [x for (x, y) in airplane_loader]
x_test = torch.cat(l, 0)
l = [y for (x, y) in airplane_loader]
y_test = torch.cat(l, 0)

!pip install git+https://github.com/fra31/auto-attack
from autoattack import AutoAttack
adversary_Linf = AutoAttack(net, norm='Linf', eps=0.031, version='standard')
adv_complete_Linf = adversary_Linf.run_standard_evaluation(x_test, y_test,bs=64)   

adv_dataset_Linf = AdversarialDataset(adv_complete_Linf, y_test)
adv_loader_Linf = torch.utils.data.DataLoader(adv_dataset_Linf, batch_size=100, shuffle=False, num_workers=2)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/fra31/auto-attack to /tmp/pip-req-build-uxxiwgyq
  Running command git clone --filter=blob:none --quiet https://github.com/fra31/auto-attack /tmp/pip-req-build-uxxiwgyq
  Resolved https://github.com/fra31/auto-attack to commit a39220048b3c9f2cca9a4d3a54604793c68eca7e
  Preparing metadata (setup.py) ... done
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.9/dist-packages/autoattack/checks.py", line 100, in check_dynamic
    sys.settrace(tracefunc)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.9/dist-packages/autoattack/checks.py", line 102, in check_dynamic
    sys.settrace(None)



initial accuracy: 8.00%
apgd-ce - 1/1 - 8 out of 8 successfully perturbed
robust accuracy after APGD-CE: 0.00% (total time 0.0 s)
max Linf perturbation: 0.00000, nan in tensor: 0, max: 1.00000, min: 0.00000
robust accuracy: 0.00%


*   to make the dataset for evaluation we mix the standard dataset with auto attack dataset and then feed it to our anomaly detection model

In [18]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        return torch.from_numpy(image), torch.tensor(label)

# Load datasets
dataset1 = x_test
dataset2 = adv_complete_Linf

# Create labels
labels1 = np.ones(dataset1.shape[0])
labels2 = np.zeros(dataset2.shape[0])

# Concatenate datasets
concatenated_dataset = np.concatenate((dataset1, dataset2), axis=0)
concatenated_labels = np.concatenate((labels1, labels2), axis=0)

# Shuffle dataset
permutation = np.random.permutation(concatenated_dataset.shape[0])
concatenated_dataset = concatenated_dataset[permutation]
concatenated_labels = concatenated_labels[permutation]

# Create dataset and dataloader
custom_dataset = CustomDataset(concatenated_dataset, concatenated_labels)
batch_size = 32
num_workers = 4
custom_dataloader = torch.utils.data.DataLoader(custom_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)


/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [19]:
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
def get_score(model, device, train_loader, test_loader):
    train_feature_space = []
    with torch.no_grad():
        for (imgs, _) in tqdm(train_loader, desc='Train set feature extracting'):
            imgs = imgs.to(device)
            features = model(imgs)
            train_feature_space.append(features)
        train_feature_space = torch.cat(train_feature_space, dim=0).contiguous().cpu().numpy()
    test_feature_space = []
    test_labels = []
    with torch.no_grad():
        for (imgs, labels) in tqdm(test_loader, desc='Test set feature extracting'):
            imgs = imgs.to(device)
            features = model(imgs)
            test_feature_space.append(features)
            test_labels.append(labels)
        test_feature_space = torch.cat(test_feature_space, dim=0).contiguous().cpu().numpy()
        test_labels = torch.cat(test_labels, dim=0).cpu().numpy()

    distances = utils.knn_score(train_feature_space, test_feature_space)

    auc = roc_auc_score(test_labels, distances)

    return auc, train_feature_space
import utils
model = utils.Model(backbone=18)
model = model.to(device)
def get_loaders(dataset, label_class, batch_size, backbone):
    if dataset == "cifar10":
        ds = custom_dataloader
        transform = transform_color if backbone == 152 else transform_resnet18
        coarse = {}
        trainset = ds(root='data', train=True, download=True, transform=transform, **coarse)
        testset = ds(root='data', train=False, download=True, transform=transform, **coarse)
        trainset_1 = ds(root='data', train=True, download=True, transform=Transform(), **coarse)
        idx = np.array(trainset.targets) == label_class
        testset.targets = [int(t != label_class) for t in testset.targets]
        trainset.data = trainset.data[idx]
        trainset.targets = [trainset.targets[i] for i, flag in enumerate(idx, 0) if flag]
        trainset_1.data = trainset_1.data[idx]
        trainset_1.targets = [trainset_1.targets[i] for i, flag in enumerate(idx, 0) if flag]
        train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2,
                                                   drop_last=False)
        test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2,
                                                  drop_last=False)
        return train_loader, test_loader, torch.utils.data.DataLoader(trainset_1, batch_size=batch_size,
                                                                      shuffle=True, num_workers=2, drop_last=False)
    else:
        print('Unsupported Dataset')
        exit()

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [20]:
train_loader, test_loader, train_loader_1 = utils.get_loaders(dataset="cifar10", label_class=7, batch_size=32, backbone=18)
a= get_score(model, device, train_loader, test_loader)
a[0]

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


Test set feature extracting: 100%|██████████| 313/313 [00:19<00:00, 15.80it/s]


0.8519885

For Label 0=airplane

In [ ]:
!python main.py --dataset=cifar10 --label=0 --backbone=18
class AdversarialDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        return sample, label

l = [x for (x, y) in airplane_loader]
x_test = torch.cat(l, 0)
l = [y for (x, y) in airplane_loader]
y_test = torch.cat(l, 0)

!pip install git+https://github.com/fra31/auto-attack
from autoattack import AutoAttack
adversary_Linf = AutoAttack(net, norm='Linf', eps=0.031, version='standard')
adv_complete_Linf = adversary_Linf.run_standard_evaluation(x_test, y_test,bs=64)   

adv_dataset_Linf = AdversarialDataset(adv_complete_Linf, y_test)
adv_loader_Linf = torch.utils.data.DataLoader(adv_dataset_Linf, batch_size=100, shuffle=False, num_workers=2)
import torch
import numpy as np

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        return torch.from_numpy(image), torch.tensor(label)

# Load datasets
dataset1 = x_test
dataset2 = adv_complete_Linf

# Create labels
labels1 = np.ones(dataset1.shape[0])
labels2 = np.zeros(dataset2.shape[0])

# Concatenate datasets
concatenated_dataset = np.concatenate((dataset1, dataset2), axis=0)
concatenated_labels = np.concatenate((labels1, labels2), axis=0)

# Shuffle dataset
permutation = np.random.permutation(concatenated_dataset.shape[0])
concatenated_dataset = concatenated_dataset[permutation]
concatenated_labels = concatenated_labels[permutation]

# Create dataset and dataloader
custom_dataset = CustomDataset(concatenated_dataset, concatenated_labels)
batch_size = 32
num_workers = 4
custom_dataloader = torch.utils.data.DataLoader(custom_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
import utils
model = utils.Model(backbone=18)
model = model.to(device)
train_loader, test_loader, train_loader_1 = utils.get_loaders(dataset="cifar10", label_class=7, batch_size=1000, backbone=18)
a= get_score(model, device, train_loader, test_loader)
a[0]

For Label 1=automobile

In [ ]:
#aslish ruie 152 bud
!python main.py --dataset=cifar10 --label=1 --backbone=18
class AdversarialDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        return sample, label

l = [x for (x, y) in automobile_loader]
x_test = torch.cat(l, 0)
l = [y for (x, y) in automobile_loader]
y_test = torch.cat(l, 0)

!pip install git+https://github.com/fra31/auto-attack
from autoattack import AutoAttack
adversary_Linf = AutoAttack(net, norm='Linf', eps=0.031, version='standard')
adv_complete_Linf = adversary_Linf.run_standard_evaluation(x_test, y_test,bs=64)   

adv_dataset_Linf = AdversarialDataset(adv_complete_Linf, y_test)
adv_loader_Linf = torch.utils.data.DataLoader(adv_dataset_Linf, batch_size=100, shuffle=False, num_workers=2)
import torch
import numpy as np

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        return torch.from_numpy(image), torch.tensor(label)

# Load datasets
dataset1 = x_test
dataset2 = adv_complete_Linf

# Create labels
labels1 = np.ones(dataset1.shape[0])
labels2 = np.zeros(dataset2.shape[0])

# Concatenate datasets
concatenated_dataset = np.concatenate((dataset1, dataset2), axis=0)
concatenated_labels = np.concatenate((labels1, labels2), axis=0)

# Shuffle dataset
permutation = np.random.permutation(concatenated_dataset.shape[0])
concatenated_dataset = concatenated_dataset[permutation]
concatenated_labels = concatenated_labels[permutation]

# Create dataset and dataloader
custom_dataset = CustomDataset(concatenated_dataset, concatenated_labels)
batch_size = 32
num_workers = 4
custom_dataloader = torch.utils.data.DataLoader(custom_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
import utils
model = utils.Model(backbone=18)
model = model.to(device)
train_loader, test_loader, train_loader_1 = utils.get_loaders(dataset="cifar10", label_class=7, batch_size=1000, backbone=18)
a= get_score(model, device, train_loader, test_loader)
a[0]

For Label 2= Bird

In [ ]:
#aslish ruie 152 bud
!python main.py --dataset=cifar10 --label=2 --backbone=18
class AdversarialDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        return sample, label

l = [x for (x, y) in bird_loader]
x_test = torch.cat(l, 0)
l = [y for (x, y) in bird_loader]
y_test = torch.cat(l, 0)

!pip install git+https://github.com/fra31/auto-attack
from autoattack import AutoAttack
adversary_Linf = AutoAttack(net, norm='Linf', eps=0.031, version='standard')
adv_complete_Linf = adversary_Linf.run_standard_evaluation(x_test, y_test,bs=64)   

adv_dataset_Linf = AdversarialDataset(adv_complete_Linf, y_test)
adv_loader_Linf = torch.utils.data.DataLoader(adv_dataset_Linf, batch_size=100, shuffle=False, num_workers=2)
import torch
import numpy as np

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        return torch.from_numpy(image), torch.tensor(label)

# Load datasets
dataset1 = x_test
dataset2 = adv_complete_Linf

# Create labels
labels1 = np.ones(dataset1.shape[0])
labels2 = np.zeros(dataset2.shape[0])

# Concatenate datasets
concatenated_dataset = np.concatenate((dataset1, dataset2), axis=0)
concatenated_labels = np.concatenate((labels1, labels2), axis=0)

# Shuffle dataset
permutation = np.random.permutation(concatenated_dataset.shape[0])
concatenated_dataset = concatenated_dataset[permutation]
concatenated_labels = concatenated_labels[permutation]

# Create dataset and dataloader
custom_dataset = CustomDataset(concatenated_dataset, concatenated_labels)
batch_size = 32
num_workers = 4
custom_dataloader = torch.utils.data.DataLoader(custom_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
import utils
model = utils.Model(backbone=18)
model = model.to(device)
train_loader, test_loader, train_loader_1 = utils.get_loaders(dataset="cifar10", label_class=7, batch_size=1000, backbone=18)
a= get_score(model, device, train_loader, test_loader)
a[0]

For Label 3= cat

In [ ]:
#aslish ruie 152 bud
!python main.py --dataset=cifar10 --label=3 --backbone=18
class AdversarialDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        return sample, label

l = [x for (x, y) in cat_loader]
x_test = torch.cat(l, 0)
l = [y for (x, y) in cat_loader]
y_test = torch.cat(l, 0)

!pip install git+https://github.com/fra31/auto-attack
from autoattack import AutoAttack
adversary_Linf = AutoAttack(net, norm='Linf', eps=0.031, version='standard')
adv_complete_Linf = adversary_Linf.run_standard_evaluation(x_test, y_test,bs=64)   

adv_dataset_Linf = AdversarialDataset(adv_complete_Linf, y_test)
adv_loader_Linf = torch.utils.data.DataLoader(adv_dataset_Linf, batch_size=100, shuffle=False, num_workers=2)
import torch
import numpy as np

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        return torch.from_numpy(image), torch.tensor(label)

# Load datasets
dataset1 = x_test
dataset2 = adv_complete_Linf

# Create labels
labels1 = np.ones(dataset1.shape[0])
labels2 = np.zeros(dataset2.shape[0])

# Concatenate datasets
concatenated_dataset = np.concatenate((dataset1, dataset2), axis=0)
concatenated_labels = np.concatenate((labels1, labels2), axis=0)

# Shuffle dataset
permutation = np.random.permutation(concatenated_dataset.shape[0])
concatenated_dataset = concatenated_dataset[permutation]
concatenated_labels = concatenated_labels[permutation]

# Create dataset and dataloader
custom_dataset = CustomDataset(concatenated_dataset, concatenated_labels)
batch_size = 32
num_workers = 4
custom_dataloader = torch.utils.data.DataLoader(custom_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
import utils
model = utils.Model(backbone=18)
model = model.to(device)
train_loader, test_loader, train_loader_1 = utils.get_loaders(dataset="cifar10", label_class=7, batch_size=1000, backbone=18)
a= get_score(model, device, train_loader, test_loader)
a[0]

For Label 4= deer

In [ ]:
#aslish ruie 152 bud
!python main.py --dataset=cifar10 --label=4 --backbone=18
class AdversarialDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        return sample, label

l = [x for (x, y) in deer_loader]
x_test = torch.cat(l, 0)
l = [y for (x, y) in deer_loader]
y_test = torch.cat(l, 0)

!pip install git+https://github.com/fra31/auto-attack
from autoattack import AutoAttack
adversary_Linf = AutoAttack(net, norm='Linf', eps=0.031, version='standard')
adv_complete_Linf = adversary_Linf.run_standard_evaluation(x_test, y_test,bs=64)   

adv_dataset_Linf = AdversarialDataset(adv_complete_Linf, y_test)
adv_loader_Linf = torch.utils.data.DataLoader(adv_dataset_Linf, batch_size=100, shuffle=False, num_workers=2)
import torch
import numpy as np

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        return torch.from_numpy(image), torch.tensor(label)

# Load datasets
dataset1 = x_test
dataset2 = adv_complete_Linf

# Create labels
labels1 = np.ones(dataset1.shape[0])
labels2 = np.zeros(dataset2.shape[0])

# Concatenate datasets
concatenated_dataset = np.concatenate((dataset1, dataset2), axis=0)
concatenated_labels = np.concatenate((labels1, labels2), axis=0)

# Shuffle dataset
permutation = np.random.permutation(concatenated_dataset.shape[0])
concatenated_dataset = concatenated_dataset[permutation]
concatenated_labels = concatenated_labels[permutation]

# Create dataset and dataloader
custom_dataset = CustomDataset(concatenated_dataset, concatenated_labels)
batch_size = 32
num_workers = 4
custom_dataloader = torch.utils.data.DataLoader(custom_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
import utils
model = utils.Model(backbone=18)
model = model.to(device)
train_loader, test_loader, train_loader_1 = utils.get_loaders(dataset="cifar10", label_class=7, batch_size=1000, backbone=18)
a= get_score(model, device, train_loader, test_loader)
a[0]

For Label 5= dog

In [ ]:
#aslish ruie 152 bud
!python main.py --dataset=cifar10 --label=5 --backbone=18
class AdversarialDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        return sample, label

l = [x for (x, y) in dog_loader]
x_test = torch.cat(l, 0)
l = [y for (x, y) in dog_loader]
y_test = torch.cat(l, 0)

!pip install git+https://github.com/fra31/auto-attack
from autoattack import AutoAttack
adversary_Linf = AutoAttack(net, norm='Linf', eps=0.031, version='standard')
adv_complete_Linf = adversary_Linf.run_standard_evaluation(x_test, y_test,bs=64)   

adv_dataset_Linf = AdversarialDataset(adv_complete_Linf, y_test)
adv_loader_Linf = torch.utils.data.DataLoader(adv_dataset_Linf, batch_size=100, shuffle=False, num_workers=2)
import torch
import numpy as np

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        return torch.from_numpy(image), torch.tensor(label)

# Load datasets
dataset1 = x_test
dataset2 = adv_complete_Linf

# Create labels
labels1 = np.ones(dataset1.shape[0])
labels2 = np.zeros(dataset2.shape[0])

# Concatenate datasets
concatenated_dataset = np.concatenate((dataset1, dataset2), axis=0)
concatenated_labels = np.concatenate((labels1, labels2), axis=0)

# Shuffle dataset
permutation = np.random.permutation(concatenated_dataset.shape[0])
concatenated_dataset = concatenated_dataset[permutation]
concatenated_labels = concatenated_labels[permutation]

# Create dataset and dataloader
custom_dataset = CustomDataset(concatenated_dataset, concatenated_labels)
batch_size = 32
num_workers = 4
custom_dataloader = torch.utils.data.DataLoader(custom_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
import utils
model = utils.Model(backbone=18)
model = model.to(device)
train_loader, test_loader, train_loader_1 = utils.get_loaders(dataset="cifar10", label_class=7, batch_size=1000, backbone=18)
a= get_score(model, device, train_loader, test_loader)
a[0]

For Label 6= frog

In [ ]:
#aslish ruie 152 bud
!python main.py --dataset=cifar10 --label=6 --backbone=18
class AdversarialDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        return sample, label

l = [x for (x, y) in frog_loader]
x_test = torch.cat(l, 0)
l = [y for (x, y) in frog_loader]
y_test = torch.cat(l, 0)

!pip install git+https://github.com/fra31/auto-attack
from autoattack import AutoAttack
adversary_Linf = AutoAttack(net, norm='Linf', eps=0.031, version='standard')
adv_complete_Linf = adversary_Linf.run_standard_evaluation(x_test, y_test,bs=64)   

adv_dataset_Linf = AdversarialDataset(adv_complete_Linf, y_test)
adv_loader_Linf = torch.utils.data.DataLoader(adv_dataset_Linf, batch_size=100, shuffle=False, num_workers=2)
import torch
import numpy as np

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        return torch.from_numpy(image), torch.tensor(label)

# Load datasets
dataset1 = x_test
dataset2 = adv_complete_Linf

# Create labels
labels1 = np.ones(dataset1.shape[0])
labels2 = np.zeros(dataset2.shape[0])

# Concatenate datasets
concatenated_dataset = np.concatenate((dataset1, dataset2), axis=0)
concatenated_labels = np.concatenate((labels1, labels2), axis=0)

# Shuffle dataset
permutation = np.random.permutation(concatenated_dataset.shape[0])
concatenated_dataset = concatenated_dataset[permutation]
concatenated_labels = concatenated_labels[permutation]

# Create dataset and dataloader
custom_dataset = CustomDataset(concatenated_dataset, concatenated_labels)
batch_size = 32
num_workers = 4
custom_dataloader = torch.utils.data.DataLoader(custom_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
import utils
model = utils.Model(backbone=18)
model = model.to(device)
train_loader, test_loader, train_loader_1 = utils.get_loaders(dataset="cifar10", label_class=7, batch_size=1000, backbone=18)
a= get_score(model, device, train_loader, test_loader)
a[0]

For Label 8= ship

In [ ]:
#aslish ruie 152 bud
!python main.py --dataset=cifar10 --label=8 --backbone=18
class AdversarialDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        return sample, label

l = [x for (x, y) in ship_loader]
x_test = torch.cat(l, 0)
l = [y for (x, y) in ship_loader]
y_test = torch.cat(l, 0)

!pip install git+https://github.com/fra31/auto-attack
from autoattack import AutoAttack
adversary_Linf = AutoAttack(net, norm='Linf', eps=0.031, version='standard')
adv_complete_Linf = adversary_Linf.run_standard_evaluation(x_test, y_test,bs=64)   

adv_dataset_Linf = AdversarialDataset(adv_complete_Linf, y_test)
adv_loader_Linf = torch.utils.data.DataLoader(adv_dataset_Linf, batch_size=100, shuffle=False, num_workers=2)
import torch
import numpy as np

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        return torch.from_numpy(image), torch.tensor(label)

# Load datasets
dataset1 = x_test
dataset2 = adv_complete_Linf

# Create labels
labels1 = np.ones(dataset1.shape[0])
labels2 = np.zeros(dataset2.shape[0])

# Concatenate datasets
concatenated_dataset = np.concatenate((dataset1, dataset2), axis=0)
concatenated_labels = np.concatenate((labels1, labels2), axis=0)

# Shuffle dataset
permutation = np.random.permutation(concatenated_dataset.shape[0])
concatenated_dataset = concatenated_dataset[permutation]
concatenated_labels = concatenated_labels[permutation]

# Create dataset and dataloader
custom_dataset = CustomDataset(concatenated_dataset, concatenated_labels)
batch_size = 32
num_workers = 4
custom_dataloader = torch.utils.data.DataLoader(custom_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
import utils
model = utils.Model(backbone=18)
model = model.to(device)
train_loader, test_loader, train_loader_1 = utils.get_loaders(dataset="cifar10", label_class=7, batch_size=1000, backbone=18)
a= get_score(model, device, train_loader, test_loader)
a[0]

For Label 9= truck

In [ ]:
#aslish ruie 152 bud
!python main.py --dataset=cifar10 --label=9 --backbone=18
class AdversarialDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        return sample, label

l = [x for (x, y) in truck_loader]
x_test = torch.cat(l, 0)
l = [y for (x, y) in truck_loader]
y_test = torch.cat(l, 0)

!pip install git+https://github.com/fra31/auto-attack
from autoattack import AutoAttack
adversary_Linf = AutoAttack(net, norm='Linf', eps=0.031, version='standard')
adv_complete_Linf = adversary_Linf.run_standard_evaluation(x_test, y_test,bs=64)   

adv_dataset_Linf = AdversarialDataset(adv_complete_Linf, y_test)
adv_loader_Linf = torch.utils.data.DataLoader(adv_dataset_Linf, batch_size=100, shuffle=False, num_workers=2)
import torch
import numpy as np

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        return torch.from_numpy(image), torch.tensor(label)

# Load datasets
dataset1 = x_test
dataset2 = adv_complete_Linf

# Create labels
labels1 = np.ones(dataset1.shape[0])
labels2 = np.zeros(dataset2.shape[0])

# Concatenate datasets
concatenated_dataset = np.concatenate((dataset1, dataset2), axis=0)
concatenated_labels = np.concatenate((labels1, labels2), axis=0)

# Shuffle dataset
permutation = np.random.permutation(concatenated_dataset.shape[0])
concatenated_dataset = concatenated_dataset[permutation]
concatenated_labels = concatenated_labels[permutation]

# Create dataset and dataloader
custom_dataset = CustomDataset(concatenated_dataset, concatenated_labels)
batch_size = 32
num_workers = 4
custom_dataloader = torch.utils.data.DataLoader(custom_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
import utils
model = utils.Model(backbone=18)
model = model.to(device)
train_loader, test_loader, train_loader_1 = utils.get_loaders(dataset="cifar10", label_class=7, batch_size=1000, backbone=18)
a= get_score(model, device, train_loader, test_loader)
a[0]